In [1]:
from debugpy.common.timestamp import current
#ставим модуль requests
!pip install requests

# ставим модуль beautifulsoup, самая последняя версия - четвертая
!pip install beautifulsoup4

# позже понадобится
!pip install fake_useragent

# прогресс выполнения
!pip install tqdm

# pandas
!pip install pandas

!pip install openpyxl

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [3]:
all_links = []

In [4]:
max_attempts = 200  # Максимальное число страниц
for counter in tqdm(range(1, max_attempts + 1)):
    url = f'https://daochai.ru/vid-chaya/page-{counter}/?result_ids=categories_view_pagination_contents&is_ajax=1'
    try:
        page = requests.get(url).json()
        content = page["html"]["categories_view_pagination_contents"]
        soup = BeautifulSoup(content, 'html.parser')
        page_links = soup.find_all('a', class_='product-title');
        for link in page_links:
            all_links.append(link['href'])
    except:
        print(f"Остановка на {counter}")
        break
else:
    print(f"Достигнут лимит ({max_attempts} страниц)")

 46%|████▌     | 91/200 [00:30<00:36,  3.01it/s]

Остановка на 92


In [8]:
all_tea = []
for chai_link in tqdm(all_links):
    current_tea = {}
    request_chai = requests.get(chai_link)
    soup_chai = BeautifulSoup(request_chai.content, 'html.parser')
    root_tag_in_chai = soup_chai.find('div', class_='cm-warehouse-block-depends-by-location')

    title = ''
    title_root = root_tag_in_chai.find('h1', class_='ut2-pb__title')
    if title_root != None:
        title = title_root.text

    current_tea['title'] = title

    description = ''
    description_tag = root_tag_in_chai.find('div', id='content_description')
    if description_tag != None:
        description = description_tag.text

    current_tea['description'] = description

    comments = []
    root_tag_for_comment = soup_chai.find('div', id='content_discussion')
    if root_tag_for_comment != None:
        for tea_comment_tag in root_tag_for_comment.findAll('div', class_='ty-discussion-post__message'):
            comments.append(tea_comment_tag.text)

    current_tea['comments'] = comments

    descriptors_final = {}
    for descriptors_root_tag in root_tag_in_chai.findAll('div', class_='descriptors-descriptor-data'):
        descriptor_key = descriptors_root_tag.find('div', class_='descriptors-descriptor-data-name')
        description_values = []
        for descriptor_value_tag in descriptors_root_tag.findAll('span', class_='descriptor-data-name'):
            description_values.append(descriptor_value_tag.text)
        descriptors_final[descriptor_key.text] = description_values

    current_tea['descriptors'] = descriptors_final

    feature_final = {}
    for feature_tag in root_tag_in_chai.findAll('div', class_='ty-product-feature'):
        feature_key = feature_tag.find('span', class_='ty-product-feature__label')
        feature_value = feature_tag.find('div', class_='ty-product-feature__value')
        feature_final[feature_key.text] = feature_value.text

    current_tea['feature'] = feature_final

    price = ''
    price_root_tag = root_tag_in_chai.find('span', class_='ty-price')
    if price_root_tag != None:
        price_tag = root_tag_in_chai.find('span', class_='ty-price-num')
        price = price_tag.text.replace('\xa0','')

    current_tea['price'] = price

    available_tea_flag = False
    available_tea = root_tag_in_chai.find('i', class_='ty-icon-ok')
    if available_tea != None:
        available_tea_flag = True

    current_tea['available_tea'] = available_tea_flag

    tea_category = []
    category_count = 0
    for tea_category_tag in root_tag_in_chai.findAll('a', class_='ty-breadcrumbs__a'):
        if category_count > 2:
            tea_category.append(tea_category_tag.text)
        category_count += 1

    current_tea['tea_category'] = tea_category

    current_tea['url'] = chai_link
    all_tea.append(current_tea)

100%|██████████| 1001/1001 [08:02<00:00,  2.07it/s]


In [9]:
tea_df = pd.DataFrame(all_tea)

In [10]:
tea_df

,title,description,comments,descriptors,feature,price,available_tea,tea_category,url
0,"Цзяньши Хуа Сянь Даньцун ""Цветочный аромат из ...",\nЭтот Дань Цун принадлежи ароматическому проф...,[],"{'Пряные': ['Мята/ментол'], 'Цветочные': ['Жас...","{'Партия:': 'июнь 2025', 'Провинция:': 'Гуанду...",1160.00,True,"[Улунский чай, Гуандунский улун (Дань цун)]",https://daochai.ru/vid-chaya/ulun/guandunskie-...
1,"Цзюань Е Найбай ""Белое молоко"", 2025 HQ",\nНевероятно красивый чай-альбинос из уезда Ан...,[],{},"{'Партия:': 'июнь 2025', 'Провинция:': 'Чжэцзя...",2277.00,True,[Зеленый чай],https://daochai.ru/vid-chaya/zeljonyj-chaj/czy...
2,"Нинхун Хунча, 2025","\nАромат сухого листа выраженно хлебный, с пло...",[],"{'Пряные': ['Гвоздика'], 'Ореховые': ['Арахис'...","{'Партия:': 'июнь 2025', 'Провинция:': 'Цзянси...",362.00,True,[Красный чай],https://daochai.ru/vid-chaya/krasnyj-chaj/ninh...
3,"Габа чай ""Изумруд"", 2025","\nЦинсин улун, прошедший процедуру ферментации...",[Буду первым. Приятная габа по приятой цене. Н...,"{'Пряные': ['Эстрагон'], 'Свежие фруктовые': [...","{'Партия:': 'июнь 2025', 'Провинция:': 'Тайван...",580.00,True,"[Улунский чай, Тайваньский улун]",https://daochai.ru/vid-chaya/ulun/taiwan-ulun/...
4,"Габа чай ""Янтарь"", 2025",\nЭта габа ароматом сухого листа напоминает “И...,[],"{'Пряные': ['Мята/ментол'], 'Цветочные': ['Пио...","{'Партия:': 'июнь 2025', 'Провинция:': 'Тайван...",821.00,True,"[Улунский чай, Тайваньский улун]",https://daochai.ru/vid-chaya/ulun/taiwan-ulun/...
...,...,...,...,...,...,...,...,...,...
996,"Шен Сяо Сигуй ""Малый Сигуй"", пресс, 2019",\nСяо Сигуй - яркий представитель шенов с мощн...,"[Мягкий шэн, не боится кипятка. Вкус свежий, з...","{'Травянистые': ['Сухая трава/солома'], 'Овощн...",{'Провинция:': 'Юньнань 云南'},9.00,False,"[Пуэр, Шен пуэр]",https://daochai.ru/vid-chaya/pu-erh/shen/shen-...
997,"Хуан Цзинь Е Дань Цун ""Золотой лист"", 2020 осень",\nНеобычный во всех отношениях даньцун от внеш...,[Вместо ДаньЦуна в пакетике оказался аньсийски...,{},{'Провинция:': 'Гуандун 广东'},18.40,False,"[Улунский чай, Гуандунский улун (Дань цун)]",https://daochai.ru/vid-chaya/ulun/guandunskie-...
998,"Моли Хуа Дань Цун ""Аромат цветов жасмина"", 202...","\nВ сухом листе маленькие жгутики, которые зам...",[вот интересно - почему так мало отзывов на эт...,{},{'Провинция:': 'Гуандун 广东'},32.00,False,"[Улунский чай, Гуандунский улун (Дань цун)]",https://daochai.ru/vid-chaya/ulun/guandunskie-...
999,"Любао Хэй Ча, корзинка, марка Ляньлюй, 2017",\nЛюбао хэйча от марки Ляньлюй (город Наньнин...,[Хороший экземпляр чая Любао. Особенно понрави...,{},{'Провинция:': 'Гуанси 广西'},10.78,False,"[Хэй ча (Тёмный чай), Любао хэйча]",https://daochai.ru/vid-chaya/heicha/lyubao-hey...


In [11]:
tea_df.to_excel('daochai_parsed.xlsx')